In [1]:
import pandas as pd
import json
import numpy as np
import ast
import locale

In [2]:
#recursos para normalizar el texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [3]:
#cargamos los datos del formato json
with open("australian_user_reviews.json", 'r', encoding="utf-8") as file:
    data_reviews= file.readlines()

# se convierten las lineas
reviews = [eval(line.strip()) for line in data_reviews]

#creamos un DF a partir de los registros

df_reviews = pd.DataFrame(reviews)

In [4]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [5]:
df_reviews2 = df_reviews['reviews'] # pasamos a un DF la columna reviews
df_reviews2.to_csv('revies_columna.csv', index=False) #convertimos la columna de reviews a csv
df_reviews2= df_reviews.explode('reviews') #quitamos corchetes []

In [6]:
df_reviews2.to_csv('revies_columna.csv', index=False)

In [14]:
df_reviews2.head()

,user_id,user_url,Posted,Item id,Helpful,Recommend,Review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,NaN,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,NaN,True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,NaN,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [8]:
# lista para anexar con el nombre en la columna reviews

funny=       []
posted=      []
last_edited= []
item_id=     []
helpful=     []
recommend=   []
review=      []

# se recorre cada fila del DF

for index, row in df_reviews2.iterrows():
    #se obtiene cada valor en la columna de 'reviews' para la fila actual
    elem= row['reviews']

    #verificamos si el valor que tenemos es un diccionario
    if isinstance(elem, dict):

        #extraemos los valores y se agregan a la lista correspondiente
        #se agrega el nombre si existe o un espacio en blanco si no existe
        funny.append(elem.get('funnr','')) 
        posted.append(elem.get('posted',''))
        last_edited.append(elem.get('last_edited',''))
        item_id.append(elem.get('item_id',''))
        helpful.append(elem.get('helpful',''))
        recommend.append(elem.get('recommend',''))
        review.append(elem.get('review',''))
    
    else:
        #de no tene el diccionario se agregan espacios en blanco
        funny.append('')
        posted.append('')
        last_edited.append('')
        item_id.append('')
        helpful.append('')
        recommend.append('')
        review.append('')

#aquí se agregan las listas como una columna nueva en el DF
df_reviews2['Funny']= funny
df_reviews2['Posted']= posted
df_reviews2['Last edited']= last_edited
df_reviews2['Item id']= item_id
df_reviews2['Helpful']= helpful
df_reviews2['Recommend']= recommend
df_reviews2['Review']= review

In [9]:
df_reviews2.drop('reviews', axis=1,inplace=True)

In [11]:
#cambiamos el valor de no ratings yet a nulos en la columna helpful

df_reviews2['Helpful']= np.where(df_reviews2['Helpful']== 'No ratings yet',np.nan, df_reviews2['Helpful'])

In [12]:
# definimos una variable para encontrar el patron de las fechas en la tabla reviews

pattern= r'Posted (\w+ \d+, \d{4})'

#filtramos y extraemos las fechas
df_reviews2['Posted']= df_reviews2['Posted'].str.extract(pattern)[0]

#convertimos las fechas en formato de date
df_reviews2['Posted']= pd.to_datetime(df_reviews2['Posted'], format= '%B %d, %Y')

In [13]:
#eliminamos columnas inecesarias 
df_reviews2.drop('Last edited', axis=1, inplace=True)
df_reviews2.drop('Funny', axis=1, inplace=True)

In [15]:
#creamos una lsita de emojis

emoji_text= {
    ':-)': 'smile',
    ';-)': 'Smile with a wink',
    ':<})': 'User with moustache, smiling',
    ':-||': 'Mad',
    ':-((': 'Sad',
    ":'-(": "Crying:~Also crying",
    ':-))': 'Really happy:-DBig grin',
    ':-*': 'A kiss',
    ':-P~': 'A lick',
    ':-o': "Wow! or I'm surprised",
    ':-|': 'Grim:-PSticking out your tongue',
    ':-/': 'perplexed',
    '=:o': 'frightened',
    ':D': 'laugh',
    ':3': 'smile',
    'xD': 'laugh loud',
    'ô•': 'oh',
    ':c': 'addictive',
    ':)': 'happy',
    ':p': 'boring',
    '^^': 'good',
    'D=': 'pain',
    ':(': 'sad',
    'xd': 'happiness',
    '=P': 'crazy',
    '<3': 'heart',
    ';>': 'blink',
    '(Y)': 'thumbs up',
    '---': 'enjoyable',
    ',3': 'amazing',
    'xt': 'unwanted',
    '-': 'recommended',
    ':}': 'good',
    'c:': 'happy',
    ':v': 'want more',
    'x3': 'cute face',
    'XD': 'addictive',
    '(y)': 'thumbs up',
    ':P': 'bad',
    ';0': 'laugh',
    '=)': 'happy',
    'gg': 'great game',
    'GG': 'great game',
    ';D': 'great game',
    ':c': 'sad',
    '~': 'snort'
}

In [16]:
#creamos una funcion para reemplazar los emojis en la columna review

def replace_emojis(text):
    for emoji, replacement in emoji_text.items():
        text = text.replace(emoji, replacement)
    return text

#aplicamos la función al data frame
df_reviews2['Review'] = df_reviews2['Review'].apply(replace_emojis)

In [24]:
# funcion para normalizar el texto 

def norm_text(t):
    #tokenizar el texto en palabras
    words= word_tokenize(t)

    #convertimos todas las palabras en minusculas
    words= [word.lower() for word in words]

    #eliminamos los signos de puntación
    words= [word for word in words if word not in string.punctuation]

    #eliminar palabras vacías
    stop_words= set(stopwords.words('english'))
    words= [word for word in words if word not in stop_words]

    #convertir palabras a su forma base
    lemmatizer= WordNetLemmatizer()
    words= [lemmatizer.lemmatize(word) for word in words]

    #unimos las palabras de nuevo en una cadena de texto
    normalized_text= ''.join(words)

    return normalized_text

#aplicar la funcion de normalizacion a la columna 'Review'
df_reviews2['normalized_review'] = df_reviews2['Review'].apply(norm_text)
